In [1]:
import pandas as pd;
import plotly.graph_objects as go
import plotly.io as pio

# https://epistat.wiv-isp.be/covid/
# Load "Hospitalisations by date and provinces" by Sciensano
hospitalisations = pd.read_csv('https://epistat.sciensano.be/Data/COVID19BE_HOSP.csv')  
lastDate = hospitalisations.iloc[[-1]]['DATE'].values[0]

regions = [{ 'column' : 'Flanders', 'type' : 'REGION', 'name' : 'Vlaams Gewest', 'population' : 6629143},
           { 'column' : 'Wallonia', 'type' : 'REGION', 'name' : 'Waals Gewest', 'population' : 3645243},
           { 'column' : 'Brussels', 'type' : 'REGION', 'name' : 'Brussels Hoofdstedelijk Gewest', 'population' : 1218255},
           { 'column' : 'OostVlaanderen', 'type' : 'PROVINCE', 'name' : 'provincie Oost-Vlaanderen', 'population' : 1525255},
           { 'column' : 'Liège', 'type' : 'PROVINCE', 'name' : 'province de Liège', 'population' : 1109800},
           { 'column' : 'VlaamsBrabant', 'type' : 'PROVINCE', 'name' : 'provincie Vlaams-Brabant', 'population' : 1155148},]
hospRegion = hospitalisations.groupby(by='DATE')
hospRegionIcu = hospRegion['TOTAL_IN_ICU'].sum()
hospRegionBasic = hospRegion['TOTAL_IN'].sum() - hospRegionIcu

fig = go.Figure( data=[
    go.Bar(name='Totaal intensieve zorg', x=hospRegionIcu.index, y=hospRegionIcu.values, textposition='auto',marker_color='rgb(50, 54, 43)'),
    go.Bar(name='Totaal niet-intensieve zorg', x=hospRegionBasic.index, y=hospRegionBasic.values, textposition='auto',marker_color='rgb(150, 180, 90)')

])
# Change the bar mode
fig.update_layout(barmode='stack',title='Covid-19 hospitalisaties België t.e.m. ' + lastDate)
fig.show(renderer='notebook_connected')
pio.write_html(fig, file='hosp/bel.html', auto_open=True)
for region in regions:

    hospRegion = hospitalisations.query(region['type'] + ' == "' + region['column'] + '"').groupby(by='DATE')
    hospRegionIcu = hospRegion['TOTAL_IN_ICU'].sum()
    hospRegionBasic = hospRegion['TOTAL_IN'].sum() - hospRegionIcu

    fig = go.Figure( data=[
        go.Bar(name='Totaal intensieve zorg', x=hospRegionIcu.index, y=hospRegionIcu.values, textposition='auto',marker_color='rgb(50, 54, 43)'),
        go.Bar(name='Totaal niet-intensieve zorg', x=hospRegionBasic.index, y=hospRegionBasic.values, textposition='auto',marker_color='rgb(150, 180, 90)')
        
    ])
    # Change the bar mode
    fig.update_layout(barmode='stack',title='Covid-19 hospitalisaties ziekenhuizen in ' + region['name'] + ' t.e.m. ' + lastDate)
    
    #population = region['population']*0.0005
    #fig.add_shape(type='line', x0='2020-03-15', y0=population, x1=lastDate, y1=population, line=dict(color='DarkOrange', width=1))
    #fig.add_annotation(text='0.05% bevolking', xanchor='left', yanchor='bottom', x='2020-03-15', y=population, showarrow=False)
    
    threeChars = region['column'][:3].lower()
    fig.show(renderer='notebook_connected')
    pio.write_html(fig, file='hosp/' + region['type'].lower() + '/' + threeChars + '.html', auto_open=True)